In [2]:
from catboost import CatBoostClassifier

import csv

from datetime import datetime

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ask")
from geopy.distance import distance
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.01, max_retries = 10)

from lightgbm import LGBMClassifier

import numpy as np 
import matplotlib.pyplot as plt
import ml_metrics as metrics
import pandas as pd

import pycountry

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingRegressor, StackingClassifier, \
GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error, average_precision_score, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from tqdm import tqdm

from xgboost import XGBClassifier

import os
for dirname, _, filenames in os.walk('Data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

Data/d.csv
Data/lat_long.csv
Data/tours_useful.csv
Data/bikers_useful.csv
Data/bikers_network_useful.csv
Data/test_tids.csv
Data/sample_submission.csv
Data/x_p.csv
Data/test_x.csv
Data/d_p.csv
Data/test_bikerids.csv
Data/x_1.csv
Data/tour_convoy.csv
Data/test.csv
Data/probabs.csv
Data/evalml_debug.log
Data/x_2.csv
Data/x_n.csv
Data/bikers.csv
Data/y_1.csv
Data/y_2.csv
Data/y.csv
Data/tours.csv
Data/x_3.csv
Data/bikers_network.csv
Data/useful.tar.xz
Data/x.csv
Data/train.csv
Data/tour_convoy_useful.csv
Data/imputed/tours_useful.csv
Data/imputed/bikers_useful.csv
Data/non_imputed/tours_useful.csv
Data/non_imputed/bikers_useful.csv
Data/khushal/training_data_final_1.csv
Data/khushal/testing_data_final_1.csv


In [3]:
train = pd.read_csv("Data/train.csv")
test = pd.read_csv("Data/test.csv")
biker = { key: row for key, *row in csv.reader(open("Data/bikers_useful.csv", 'r'))}
print(biker.pop('biker_id'))
biker_net = { key: row for key, *row in csv.reader(open("Data/bikers_network_useful.csv", 'r'))}
print(biker_net.pop('biker_id'))
tours = { key: row for key, *row in csv.reader(open("Data/tours_useful.csv", 'r'))}
print(tours.pop('tour_id'))
tour_convoy = { key: row for key, *row in csv.reader(open("Data/tour_convoy_useful.csv", 'r'))}
print(tour_convoy.pop('tour_id'))

['language_id', 'location_id', 'bornIn', 'gender', 'member_since', 'area', 'time_zone', 'latitude', 'longitude']
['friends']
['biker_id', 'tour_date', 'city', 'state', 'pincode', 'country', 'latitude', 'longitude', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'w12', 'w13', 'w14', 'w15', 'w16', 'w17', 'w18', 'w19', 'w20', 'w21', 'w22', 'w23', 'w24', 'w25', 'w26', 'w27', 'w28', 'w29', 'w30', 'w31', 'w32', 'w33', 'w34', 'w35', 'w36', 'w37', 'w38', 'w39', 'w40', 'w41', 'w42', 'w43', 'w44', 'w45', 'w46', 'w47', 'w48', 'w49', 'w50', 'w51', 'w52', 'w53', 'w54', 'w55', 'w56', 'w57', 'w58', 'w59', 'w60', 'w61', 'w62', 'w63', 'w64', 'w65', 'w66', 'w67', 'w68', 'w69', 'w70', 'w71', 'w72', 'w73', 'w74', 'w75', 'w76', 'w77', 'w78', 'w79', 'w80', 'w81', 'w82', 'w83', 'w84', 'w85', 'w86', 'w87', 'w88', 'w89', 'w90', 'w91', 'w92', 'w93', 'w94', 'w95', 'w96', 'w97', 'w98', 'w99', 'w100', 'w_other']
['going', 'maybe', 'invited', 'not_going']


In [4]:
train_bikers_set = np.array(train["biker_id"].drop_duplicates())
test_bikers_set = np.array(test["biker_id"].drop_duplicates())
req_bikers_set = np.array([*biker])
tours_set = np.array([*tours])
req_bikers_set.shape, train_bikers_set.shape, tours_set.shape

((2231,), (1934,), (9343,))

In [5]:
tour_convoy_full = { key: row for key, *row in csv.reader(open("Data/tour_convoy.csv", 'r'))}

In [6]:
events_attended = {}
events_notgoing = {}
events_maybe = {}
for tid, row in tour_convoy_full.items():
    people_attended = row[0].split()
    people_maybe = row[1].split()
    people_notgoing = row[2].split()
    for bid in people_attended:
        events_attended[bid] = events_attended.get(bid, 0) + 1
    for bid in people_notgoing:
        events_notgoing[bid] = events_notgoing.get(bid, 0) + 1
    for bid in people_maybe:
        events_maybe[bid] = events_maybe.get(bid, 0) + 1
        
events_attended_friends = {}
events_notgoing_friends = {}
events_maybe_friends = {}
no_friends = {}
for bid, row in biker_net.items():
    friends = row[0].split()
    no_friends[bid] = len(friends)
    for friend in friends:
        events_attended_friends[bid] = events_attended_friends.get(bid, 0) + events_attended.get(friend, 0)
        events_notgoing_friends[bid] = events_notgoing_friends.get(bid, 0) + events_notgoing.get(friend, 0)
        events_maybe_friends[bid] = events_maybe_friends.get(bid, 0) + events_maybe.get(friend, 0)
        
events_attended_friends

{'IF1286067': 10,
 'EB01519751': 54,
 'CB63304477': 25,
 'DJ7493741': 14,
 'DI3523902': 75,
 'DF54343061': 8,
 'BF38685648': 150,
 'DC51568850': 32,
 'CI79905973': 52,
 'DA95718598': 8,
 'CA08761267': 253,
 'EI3332100': 38,
 'DA71657222': 36,
 'BJ48333652': 21,
 'BI72064755': 0,
 'DB42327997': 59,
 'CH52875123': 5,
 'EC88238269': 5,
 'CD56937947': 116,
 'EA58274528': 23,
 'BD69905557': 54,
 'BI09934947': 28,
 'BF07405031': 29,
 'BG29404827': 15,
 'FB7270916': 128,
 'DD25600533': 73,
 'DI87778002': 12,
 'DB10482182': 61,
 'FG9826712': 103,
 'HF47671': 8,
 'DJ25613292': 20,
 'CA48999821': 19,
 'DB23500032': 195,
 'CG37352610': 15,
 'BA79339663': 1429,
 'DG46961870': 92,
 'CC71261888': 12,
 'DI16191065': 140,
 'DJ86193980': 90,
 'CB93694254': 47,
 'CE7769793': 83,
 'BJ18945522': 30,
 'CB87183134': 56,
 'CJ71889961': 85,
 'CF56133275': 20,
 'CD93365897': 7,
 'DH09066919': 21,
 'EB79324570': 23,
 'BH74655165': 31,
 'CB64973127': 146,
 'FC7523423': 16,
 'DB31909049': 48,
 'BC92184871': 25,
 

In [7]:
tours_full = pd.read_csv("Data/tours.csv").values
len_tf = tours_full.shape[0]
np.random.seed(42)
tours_full = tours_full[np.random.choice(len_tf, len_tf//10)]
tours_full.shape

(313797, 110)

In [8]:
#Cluster word frequencies using K Means
#word_count = tours_full.loc[:, "w1":"w_other":1].values
word_count = []
for row in tqdm(tours_full):
    w = []
    for i in range(9, 110):
        w.append(int(row[i]))
    word_count.append(w)
    
word_count = np.array(word_count)
 
kmeans = KMeans(n_clusters=30, random_state=0).fit(word_count)
#kmeans = MiniBatchKMeans(n_clusters=30, batch_size = 10, random_state=0).fit(tours_full)
#tours_full = None

100%|██████████| 313797/313797 [00:11<00:00, 28471.48it/s]


In [9]:
word_count.shape

(313797, 101)

In [10]:
#Selecting top countries for bikers and putting rest as others
countries = {}
for bid, row in biker.items():
    countries[row[1]] = countries.get(row[1], 0) + 1
    
countries_list = list(countries.keys())
countries_list.sort(key = lambda x : countries[x], reverse = True)
top_10_countries = countries_list[:10]
country_val = {}
for i, country in enumerate(top_10_countries):
    country_val[country] = i
    
#Selecting top countries for tours and putting rest as others
tour_countries = {}
for bid, row in tours.items():
    tour_countries[row[5]] = tour_countries.get(row[5], 0) + 1
    
tour_countries_list = list(tour_countries.keys())
tour_countries_list.sort(key = lambda x : tour_countries[x], reverse = True)
tour_top_10_countries = tour_countries_list[:10]
tour_country_val = {}
for i, country in enumerate(tour_top_10_countries):
    tour_country_val[country] = i    
    
#Selecting top cities for tours and putting rest as others
tour_cities = {}
for bid, row in tours.items():
    tour_cities[row[2]] = tour_cities.get(row[2], 0) + 1
    
tour_cities_list = list(tour_cities.keys())
tour_cities_list.sort(key = lambda x : tour_cities[x], reverse = True)
tour_top_10_cities = tour_cities_list[:10]
tour_city_val = {}
for i, city in enumerate(tour_top_10_cities):
    tour_city_val[city] = i    

#Selecting top languages for bikers and putting rest as others
langs = {}
for bid, row in biker.items():
    langs[row[1]] = langs.get(row[0], 0) + 1
    
langs_list = list(langs.keys())
langs_list.sort(key = lambda x : langs[x], reverse = True)
top_10_langs = langs_list[:10]
lang_val = {}
for i, lang in enumerate(top_10_langs):
    lang_val[lang] = i

top_10_langs

['ID', 'ES', 'FR', 'KR', 'US', 'AR', 'GB', 'LA', 'GE', 'CA']

In [11]:
top_10_loc = { 
                 'ID': [-2.4833826, 117.8902853],
                 'US': [39.7837304, -100.4458825],
                 'GB': [54.7023545, -3.2765753],
                 'LA': [20.0171109, 103.378253],
                 'ES': [39.3262345, -4.8380649],
                 'FR': [46.603354, 1.8883335],
                 'GE': [32.3293809, -83.1137366],
                 'AR': [-34.9964963, -64.9672817],
                 'CN': [35.000074, 104.999927],
                 'RU': [64.6863136, 97.7453061]
}
for bid, row in biker.items():
    if row[7] == '0' and row[1] in top_10_loc:
        biker[bid][7] = top_10_loc[row[1]][0]
    if row[8] == '0' and row[1] in top_10_loc:
        biker[bid][8] = top_10_loc[row[1]][1]     


In [12]:
biker_full = { key: row for key, *row in csv.reader(open("Data/bikers.csv", 'r'))}
print(biker_full.pop('biker_id'))

['language_id', 'location_id', 'bornIn', 'gender', 'member_since', 'area', 'time_zone']


In [13]:
biker_full_lat = {}
biker_full_long = {}
for bid, row in biker_full.items():
    if row[1] in top_10_loc:
        biker_full_lat[bid] = top_10_loc[row[1]][0]
        biker_full_long[bid] = top_10_loc[row[1]][1]

In [14]:
cc_set = {
            'LA': [20.0171109, 103.378253],
            'RU': [64.6863136, 97.7453061],
            'ID': [-2.4833826, 117.8902853],
            'LT': [55.3500003, 23.7499997],
            'VN': [13.2904027, 108.4265113],
            'CN': [35.000074, 104.999927],
            'BR': [-10.3333333, -53.2],
            'ES': [39.3262345, -4.8380649],
            'FR': [46.603354, 1.8883335],
            'UD': [39.7837304, -100.4458825],
            'US': [39.7837304, -100.4458825],
            'GE': [32.3293809, -83.1137366],
            'GB': [54.7023545, -3.2765753],
            'TW': [23.9739374, 120.9820179],
            'CA': [61.0666922, -107.9917071]
        }

com_coun = {**cc_set, **top_10_loc}

def get_nan_count():
    nan_count = 0
    for tid, row in tours.items():
        if row[6] == '' or row[7] == '':
            nan_count+=1
    return nan_count

def get_friend_org(org):
    for bid, row in biker_net.items():
        friends = row[0].split()
        if org in friends:
            return bid
    return ''


def repl_missing_tour_loc1():
    for tid, row in tours.items():
        if row[6] == '':
            att_list = tour_convoy_full[tid][0].split()
            bc_list = []
            for bi in att_list:
                if bi in biker_full:
                    bc_list.append(biker_full[bi][1])
            if len(bc_list) > 0:
                most_common_c = max(set(bc_list), key=bc_list.count)
                #print(most_common_c)
                tours[tid][6] = com_coun[most_common_c][0]
        if row[7] == '':
            att_list = tour_convoy_full[tid][0].split()
            bc_list = []
            for bi in att_list:
                if bi in biker_full:
                    bc_list.append(biker_full[bi][1])
            if len(bc_list) > 0:
                most_common_c = max(set(bc_list), key=bc_list.count)
                tours[tid][7] = com_coun[most_common_c][1]

def repl_missing_tour_loc2():
    for tid, row in tqdm(tours.items()):
        fren = ''
        if row[6] == '' or row[7] == '':
            fren = get_friend_org(row[0])
        if row[6] == '':
            if fren not in biker_full_lat:
                continue
            #print("hi")
            tours[tid][6] = biker_full_lat[fren]
        if row[7] == '':
            if fren not in biker_full_long:
                continue
            tours[tid][7] = biker_full_long[fren]

repl_missing_tour_loc1()
repl_missing_tour_loc2()

latlist = []
longlist = []
for tid, row in tours.items():
    if row[6] != '':
        latlist.append((float(row[6])))
    if row[7] != '':
        longlist.append((float(row[7])))
        
latmed = np.median(np.array(latlist))
longmed = np.median(np.array(longlist))

for tid, row in tours.items():
    if row[6] == '':
        tours[tid][6] = latmed
    if row[7] == '':
        tours[tid][7] = longmed
get_nan_count()

100%|██████████| 9343/9343 [07:16<00:00, 21.42it/s]


0

In [15]:
#Number of tours each person has in train/test set
train_count = {}
test_count = {}
like_count = {}
dislike_count = {}
train_tour_count = {}
test_tour_count = {}

for index, row in train.iterrows():
    train_count[row["biker_id"]]   = train_count.get(row["biker_id"],0)+1
    like_count[row["biker_id"]]    = like_count.get(row["biker_id"],0)+row["like"]
    dislike_count[row["biker_id"]] = dislike_count.get(row["biker_id"],0)+row["dislike"]
    train_tour_count[row["tour_id"]]   = train_tour_count.get(row["tour_id"],0)+1
    
    
for index, row in test.iterrows():
    test_count[row["biker_id"]] = test_count.get(row["biker_id"],0)+1
    test_tour_count[row["tour_id"]]   = test_tour_count.get(row["tour_id"],0)+1

In [68]:
events_attended_dist = {}
events_notgoing_dist = {}
events_maybe_dist = {}
for tid, row in tour_convoy_full.items():
    if tid == 'tour_id' or tid not in tours:
        continue
    people_attended = row[0].split()
    people_maybe = row[1].split()
    people_notgoing = row[2].split()
    for bid in people_attended:
        events_attended_dist[bid] = events_attended_dist.get(bid, np.zeros(101)) + np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    for bid in people_notgoing:
        events_notgoing_dist[bid] = events_notgoing_dist.get(bid, np.zeros(101)) + np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    for bid in people_maybe:
        events_maybe_dist[bid] = events_maybe_dist.get(bid, np.zeros(101)) + np.array(list(map(lambda x: int(x), tours[tid][8:109])))
        
events_attended_friends_dist = {}
events_notgoing_friends_dist = {}
events_maybe_friends_dist = {}
for bid, row in biker_net.items():
    friends = row[0].split()
    for friend in friends:
        events_attended_friends_dist[bid] = events_attended_friends_dist.get(bid, np.zeros(101)) + events_attended_dist.get(friend, np.zeros(101))
        events_notgoing_friends_dist[bid] = events_notgoing_friends_dist.get(bid, np.zeros(101)) + events_notgoing_dist.get(friend, np.zeros(101))
        events_maybe_friends_dist[bid] = events_maybe_friends_dist.get(bid, np.zeros(101)) + events_maybe_dist.get(friend, np.zeros(101))
        
for bid in events_attended_dist:
    events_attended_dist[bid] /= (events_attended[bid] + 1)
    
for bid in events_notgoing_dist:
    events_notgoing_dist[bid] /= (events_notgoing[bid] + 1)
    
for bid in events_maybe_dist:
    events_maybe_dist[bid] /= (events_maybe[bid] + 1)
    
for bid in events_attended_friends_dist:
    events_attended_friends_dist[bid] /= (events_attended_friends[bid] + 1)
    
for bid in events_notgoing_friends_dist:
    events_notgoing_friends_dist[bid] /= (events_notgoing_friends[bid] + 1)
    
for bid in events_maybe_friends_dist:
    events_maybe_friends_dist[bid] /= (events_maybe_friends[bid] + 1)
    
len(events_attended_friends_dist)

2231

In [83]:
def get_like_count(bid):
    return like_count.get(bid,0)

def get_dislike_count(bid):
    return dislike_count.get(bid,0)

def get_no_friends_count(bid):
    return no_friends.get(bid,0)

def get_train_count(bid):
    return train_count.get(bid,0)

def get_test_count(bid):
    return test_count.get(bid,0)

def get_train_tour_count(tid):
    return train_tour_count.get(tid,0)

def get_test_tour_count(tid):
    return test_tour_count.get(tid,0)

def get_country(bid):
    return country_val.get(biker[bid][1], 10)

def get_lang(bid):
    return lang_val.get(biker[bid][0].upper(), 10)

def get_tour_country(tid):
    return tour_country_val.get(tours[tid][5], 10)

def get_tour_city(tid):
    return tour_city_val.get(tours[tid][2], 10)

def get_delta1(bid, tid, timestamp):
    return (abs(datetime.strptime(timestamp[:10], "%d-%m-%Y") - 
                datetime.strptime(tours[tid][1], "%d-%m-%Y")).total_seconds())/1e7

def get_delta2(bid, tid, timestamp):
    return (abs(datetime.strptime(timestamp[:10], "%d-%m-%Y") - 
                datetime.strptime(biker[bid][4], "%d-%m-%Y")).total_seconds())/1e7

def get_delta3(bid, tid):
    return (abs(datetime.strptime(biker[bid][4], "%d-%m-%Y") - 
                datetime.strptime(tours[tid][1], "%d-%m-%Y")).total_seconds())/1e7

def get_distance(bid, tid):
    return distance((biker[bid][7], biker[bid][8]), (tours[tid][6], tours[tid][7])).miles

def get_top10_wc(tid):
    row = tours[tid]
    w = []
    for i in range(8, 109):
        w.append(int(row[i]))
    w = np.array(w)
    return kmeans.predict(np.array([w]))[0]

def get_sum_wc(tid):
    row = tours[tid]
    s = 0
    for i in range(8,18):
        s += int(row[i])
    return s

def get_gender(bid):
    return biker[bid][3] == 'male'
    
def get_age(bid):
    by = (biker[bid][2])
    if by == None or by == 'None':
        by = 1990
    return (2020 - int(by))

def get_friend_with_org(bid, tid):
    org = tours[tid][0]
    frens = biker_net[bid][0].split()
    return org in frens

def get_ymdw(tobj):
    tim = datetime.strptime(tobj, "%d-%m-%Y")
    return [tim.year, tim.month, tim.day, tim.weekday()]

def get_timezone(bid):
    tz = biker[bid][6]
    if tz == '':
        return 7.0
    return int(tz)/60

def get_sim_a(bid, tid):
    v1 = events_attended_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_n(bid, tid):
    v1 = events_notgoing_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_m(bid, tid):
    v1 = events_maybe_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_af(bid, tid):
    v1 = events_attended_friends_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_nf(bid, tid):
    v1 = events_notgoing_friends_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

def get_sim_mf(bid, tid):
    v1 = events_maybe_friends_dist.get(bid, None)
    if v1 is None:
        return 0
    v2 = np.array(list(map(lambda x: int(x), tours[tid][8:109])))
    a = np.linalg.norm(v1)
    b = np.linalg.norm(v2)
    if a == 0 or b == 0 :
        return 0
    return np.dot(v1, v2)/(a*b)

fraction_in = {}
total_attendees = {}
deltas = {}
total_maybe = {}
total_notgoing = {}

for tid, row in tour_convoy.items():
    not_going = len(row[3].split())
    invited = len(row[2].split())
    maybe = len(row[1].split())
    attend = len(row[0].split())
    fraction_in[tid] = (not_going)/(invited+1)
    total_attendees[tid] = attend
    total_maybe[tid] = maybe
    total_notgoing[tid] = maybe

for index, row in train.iterrows():
    a = row["biker_id"]
    b = row["tour_id"]
    c = row["timestamp"]
    curr_delta = get_delta1(a,b,c)
    if a not in deltas:
        deltas[a] = curr_delta
    else:
        deltas[a] = min(deltas[a], curr_delta)
        
for index, row in test.iterrows():
    a = row["biker_id"]
    b = row["tour_id"]
    c = row["timestamp"]
    curr_delta = get_delta1(a,b,c)
    if a not in deltas:
        deltas[a] = curr_delta
    else:
        deltas[a] = min(deltas[a], curr_delta)

In [231]:
d = 0
def make_feature(bid, tid, timestamp, invited):
    f = [0.0]*224
    
    f[0] = get_delta1(bid, tid, timestamp)
    f[1] = fraction_in[tid]
    f[2] = total_attendees[tid]
    f[3] = get_distance(bid, tid)
    f[4] = events_attended_friends[bid]
    f[5] = 1 if f[0] == 0 else deltas[bid]/f[0]
    f[6] = invited
    f[7] = total_maybe[tid]
    f[8] = events_notgoing_friends[bid]
    f[9] = events_maybe_friends[bid]
    
    country = get_country(bid)
    for i in range(11):
        f[10+i] = int(i == country)
    
    wc = get_top10_wc(tid)
    for i in range(30):
        f[21+i] = int(i == wc)
    
    f[51] = get_train_count(bid)
    f[52] = get_test_count(bid)
    f[53] = get_delta2(bid, tid, timestamp)
    f[54] = get_delta3(bid, tid)
    f[55] = total_notgoing[tid]
    f[56] = get_gender(bid)
    f[57] = get_age(bid)
    f[58] = get_friend_with_org(bid, tid)
    
    t1 = get_ymdw(timestamp[:10])
    t1 += get_ymdw(biker[bid][4])
    t1 += get_ymdw(tours[tid][1])
    for j in range(12):
        f[59+j] = t1[j]
    
    tour_country = get_tour_country(tid)
    for i in range(11):
        f[71+i] = int(i == tour_country)
    
    tour_city = get_tour_city(tid)
    for i in range(11):
        f[82+i] = int(i == tour_city)
    
    lang = get_lang(bid)
    for i in range(11):
        f[93+i] = int(i == lang)
    
    f[104] = get_timezone(bid)
    f[105] = get_sum_wc(tid)
    f[106] = (t1[3] in [4,5,6])
    f[107] = (t1[7] in [4,5,6])
    f[108] = (t1[11] in [4,5,6])
    
    f[109] = get_like_count(bid)
    f[110] = get_dislike_count(bid)
    f[111] = get_no_friends_count(bid)
    
    f[112] = events_attended.get(bid,0)
    f[113] = events_maybe.get(bid,0)
    f[114] = events_notgoing.get(bid,0)
    
    for i in range(8, 109):
        f[115+i-8] = int(tours[tid][i])
        
    f[216] = get_train_tour_count(tid)
    f[217] = get_test_tour_count(tid)
    
    f[218] = get_sim_a(bid, tid)
    f[219] = get_sim_n(bid, tid)
    f[220] = get_sim_m(bid, tid)
    f[221] = get_sim_af(bid, tid)
    f[222] = get_sim_nf(bid, tid)
    f[223] = get_sim_mf(bid, tid)
    
    return np.array(f)

X, Y = [], []
for index, row in train.iterrows():
    a = row["biker_id"]
    b = row["tour_id"]
    c = row["timestamp"]
    d = row["invited"]
    X.append(make_feature(a, b, c, d))
    if(row["like"] == 1):
        Y.append(1)
    else:
        Y.append(0)
        
X = np.array(X)
Y = np.array(Y)

#scaler = StandardScaler()
#scaler.fit(X)
#X = scaler.transform(X)
test_X = []
for index, row in test.iterrows():
    a = row["biker_id"]
    b = row["tour_id"]
    c = row["timestamp"]
    d = row["invited"]
    test_X.append(make_feature(a, b, c, d))

        
test_X = np.array(test_X)

X.shape

(13866, 224)

In [248]:
ts = {}
inv = {}
for index, row in train.iterrows():
    ts[(row["biker_id"], row["tour_id"])] = row["timestamp"]
    inv[(row["biker_id"], row["tour_id"])] = row["invited"]

def get_ground(data):
    biker_set_curr = np.array(data["biker_id"].drop_duplicates())
    index = {}
    for i, biker in enumerate(biker_set_curr):
        index[biker]=i
    result = [[]]*len(biker_set_curr)
    for id, row in data.iterrows():
        if row["like"] == 1:
            #result[index[row["biker_id"]]].append(row["tour_id"])
            result[index[row["biker_id"]]].append(1)
        else:
            result[index[row["biker_id"]]].append(0)
    return result

def get_pred(data, classifier, reg):
    biker_set_curr = np.array(data["biker_id"].drop_duplicates())
    tours_out =[[]]*len(biker_set_curr)
    index = {}
    for i, biker in enumerate(biker_set_curr):
        index[biker]=i
    for id, row in data.iterrows():
        biker1 = row["biker_id"]
        tou = row["tour_id"]
        fe = make_feature(biker1, tou, ts[(biker1, tou)], inv[(biker1, tou)]).reshape(1,-1)
        s = classifier.predict_proba(fe)[0][1]
        tours_out[index[biker1]].append(s)
    return tours_out

def cross_validate(X, Y, classifier, reg=False):
    kf = KFold(n_splits=4)
    scores = []
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        classifier.fit(X_train, y_train)
        test_data = train.iloc[test_index]
        ground_truth = get_ground(test_data)
        predictions = get_pred(test_data, classifier, reg)
        #print((ground_truth), (predictions))
        curr_score = 0.0
        for gt, pre in zip(ground_truth, predictions):
            #print(gt, pre)
            curr_score += average_precision_score(gt, pre)
        curr_score /= len(ground_truth)
        scores.append(curr_score)
    return np.array(scores)        


nf = X.shape[1]
rfmf = nf - 2
gbmf = nf - 1
#{'learning_rate': 0.11664380912238853, 'max_depth': 2, 'colsample_bylevel': 1.0, 'scale_pos_weight': 1.0, 'bagging_temperature': 43.095488904563325,
# 'random_strength': 54.31426569199799, 'reg_lambda': 73.12682176174783}
classifiers = [
    #RandomForestClassifier(max_depth=20, n_estimators=201, max_features=rfmf, max_samples = 0.7, random_state = 0, criterion='entropy'),
    #XGBClassifier(max_depth=15, n_estimators=201,  random_state = 0, verbosity = 0, use_label_encoder=False, objective = 'binary:logistic', 
    #              min_child_weight=3,learning_rate=0.05,gamma=0.3, colsample_bytree=0.7),
    LGBMClassifier(max_depth=-1, n_estimators=201,  random_state = 0, verbosity = 0, subsample=0.9, reg_lambda = 0.00043856,
                  learning_rate=0.1,num_leaves=59, colsample_bytree=0.9, force_row_wise=True),
    #AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),algorithm="SAMME",n_estimators=351),
    #MLPClassifier(random_state=1, max_iter=500, solver='adam'),{'depth': 6, 'l2_leaf_reg': 11, 'iterations': 200, 'learning_rate': 0.1}
    CatBoostClassifier(iterations=250, learning_rate=0.10001, depth=11, eval_metric='Accuracy', random_state=0, verbose=249, l2_leaf_reg = 3.0121),
#     CatBoostClassifier(iterations=250, eval_metric='Accuracy', random_state=0, verbose=249, learning_rate = 0.11664380912238853, max_depth = 2,
#                        colsample_bylevel = 1.0, scale_pos_weight = 1.0, bagging_temperature = 43.095488904563325, random_strength = 54.31426569199799, 
#                        reg_lambda = 73.12682176174783),
    GradientBoostingClassifier(max_depth=15, n_estimators=201, max_features=gbmf, random_state = 0)
]
clf0 = classifiers[0]
clf1 = classifiers[1]
#clf2 = classifiers[2]
# scores = cross_validate(X , Y, clf0, False)
# scores = cross_val_score(clf0, X, Y, cv = 4, scoring = 'accuracy')
# print(scores)
# print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
# #scores = cross_validate(X, Y, clf1, True)
# scores = cross_val_score(clf1, X, Y, cv = 4, scoring = 'accuracy')
# print(scores)
# print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
# clf0.fit(X,Y)
# clf1.fit(X,Y)
# clf3=BaggingClassifier(n_estimators=17,base_estimator=clf0,random_state=0)
# clf4=BaggingClassifier(n_estimators=17,base_estimator=clf1,random_state=0)
# clf5=BaggingClassifier(n_estimators=17,base_estimator=clf1,random_state=0)
clf3=AdaBoostClassifier(base_estimator=clf0,n_estimators=130,random_state=0)
clf4=AdaBoostClassifier(base_estimator=clf1,n_estimators=130,random_state=0)
clf5=AdaBoostClassifier(base_estimator=clf1,n_estimators=130,random_state=0)
weights = [0.3,0.3,0.3]
clf = VotingClassifier(estimators=[('rf',clf3), 
                                   ('cb',clf4), 
                                   ('gb', clf5)
                                  ], weights = weights, voting ='soft')
#clf=AdaBoostClassifier(base_estimator=clf3,n_estimators=5,random_state=0)
#clf=BaggingClassifier(n_estimators=9,base_estimator=clf0,random_state=0)

# scores = cross_val_score(clf, X, Y, cv = 4, scoring = 'accuracy')
# print(scores)
# print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

## clf.fit(X,Y)

# estimators = [
#     ('rf', RandomForestClassifier(max_depth=20, n_estimators=151, max_features=rfmf, max_samples = 0.7, random_state = 0)),
#     ('ab', AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),algorithm="SAMME",n_estimators=351))
#     #('nn', MLPClassifier(random_state=1, max_iter=500, solver='adam')),
#     #('cb', CatBoostClassifier(depth = 15, num_trees=201, random_state = 0, early_stopping_rounds = 20, verbose=50)),
#  ]
# estimators1 = [
#     ('gb',GradientBoostingClassifier(max_depth=15, n_estimators=201, random_state = 0)),
#     #('xg', XGBClassifier(max_depth=15, n_estimators=201,  random_state = 0, verbosity = 1, use_label_encoder=False))
#     ('rf', RandomForestClassifier(max_depth=20, n_estimators=201, random_state = 0))
# ]
# clf2 = StackingClassifier(
#      estimators=estimators1, final_estimator=LogisticRegression()
#  )
# clf = StackingClassifier(
#      estimators=estimators, final_estimator=clf2
#  )
# scores = cross_val_score(clf, X, Y, cv = 4, scoring = 'accuracy')
# print(scores)
# print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))
# clf.fit(X,Y)

# model = CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy', random_state=0)
# grid = {'learning_rate': [0.03, 0.1],
#         'depth': [6, 10, 15],
#         'l2_leaf_reg': [3, 5, 7, 9, 11],
#         'iterations': [200, 300, 400]
#        }

# grid_search_result = model.grid_search(grid, 
#                                        X=X, 
#                                        y=Y, 
#                                        plot=True)
# rfc=RandomForestClassifier(random_state=0)

# param_grid = { 
#     'n_estimators': [501],
#     'max_features': [rfmf],
#     'max_depth' : [14],
#     'criterion' :['entropy'],
#     'max_samples':[0.5,0.7,0.9]
# }

# CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, verbose = 2, n_jobs = 4)
# CV_rfc.fit(X, Y)

# param_grid = { 
#         'n_estimators':[13,17,21]
# }
# model = BaggingClassifier(base_estimator=clf0,random_state=0)
# CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5, verbose = 2, n_jobs = 4)
# CV_rfc.fit(X, Y)

# def rc(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, lr, num_leaves1,max_depth1,lambda_l21,min_child_samples1, lr1):
#     params = {
#         "objective" : "binary",
#         "metric" : "auc", 
#         'is_unbalance': True,
#         "num_leaves" : int(num_leaves),
#         "max_depth" : int(max_depth),
#         "reg_alpha" : lambda_l2,
#         "reg_lambda" : lambda_l1,
#         "n_jobs" : 20,
#         "min_child_samples" : int(min_child_samples),
#         "learning_rate" : lr,
#         "subsample_freq" : 5,
#         "bagging_seed" : 42,
#         "verbosity" : -1,
#         "force_row_wise" : True,
#         "random_state" : 0
#     }
#     params1 = {
#         "eval_metric" : "AUC", 
#         #"num_leaves" : int(num_leaves1),
#         "max_depth" : int(max_depth1),
#         "l2_leaf_reg" : lambda_l21,
#         "min_child_samples" : int(min_child_samples1),
#         "learning_rate" : lr1,
#         "verbose" : False,
#         "random_state" : 0,
#         "iterations" : 250
#     }
#     cll = LGBMClassifier(**params)
#     clc = CatBoostClassifier(**params1)
# #     clg3=AdaBoostClassifier(base_estimator=cll,n_estimators=100,random_state=0)
#     clg4=AdaBoostClassifier(base_estimator=clc,n_estimators=100,random_state=0)
#     clg5=AdaBoostClassifier(base_estimator=clc,n_estimators=100,random_state=0)
#     weights = [0.3,0.3,0.3]
#     clg = VotingClassifier(estimators=[ ('lg',cll), 
#                                         ('cb',clg4),
#                                         ('cb1', clg5)
#                                   ], weights = weights, voting ='soft')
#     return clc

# params = {'lambda_l1': 0.16570937758584275, 'lambda_l2': 0.40140411614102267, 'lambda_l21': 4.0, 'lr': 0.1292823676954809, 
#           'lr1': 0.0650733105002455, 'max_depth': 19.82527142156306, 'max_depth1': 6.2, 'min_child_samples': 6885.727758041044, 
#           'min_child_samples1': 18, 'num_leaves': 53.0, 'num_leaves1': 65}

params = {'max_depth': 8, 'num_leaves': 80, 'min_child_samples': 200, 'scale_pos_weight': 1.0, 'subsample': 0.6303896252329312, 'colsample_bytree': 0.6, 
          'reg_alpha': 3.3388399958654817, 'n_jobs': 8, 'boosting_type': 'gbdt', 'objective': 'binary', 
          'learning_rate': 0.07677467484724665, 'max_bin': 100, 'random_state': 0, 'verbose': -1}

def rc(values):
    params = {'max_depth': values[0],
              'num_leaves': values[1],
              'min_child_samples': values[2],
              'scale_pos_weight': values[7],
              'subsample': values[3],
              'colsample_bytree': values[4],
              'reg_alpha': values[6],
              'n_jobs': 8,
              'boosting_type': 'gbdt',
              'objective': 'binary',
              'learning_rate': values[5],
              'max_bin': 100,
              #              'min_child_weight': 0,
              #              'min_split_gain': 0,
              #              'subsample_freq': 1,
              'random_state' : 0,
              'verbose' : -1
              }
    model = LGBMClassifier(
        **params
    )
    return model

#clf = rc(list(params.values()))
#clf1 = LGBMClassifier(random_state = 0, verbose=-1, num_leaves = 45, reg_alpha = 1, reg_lambda = 0.990,
#                   learning_rate = 0.09116, min_child_samples = 15)
# clf2 = LGBMClassifier(random_state = 0, verbose=-1, num_leaves = 60, reg_alpha = 0.027870894746092358, reg_lambda = 0.009357684241550774,
#                    learning_rate = 0.09116640670855684, min_child_samples = 25)
# clf3 = LGBMClassifier(random_state = 0, verbose=-1, num_leaves = 75, reg_alpha = 0.027870894746092358, reg_lambda = 0.009357684241550774,
#                    learning_rate = 0.09116640670855684, min_child_samples = 35)
# clf4 = LGBMClassifier(random_state = 0, verbose=-1, num_leaves = 90, reg_alpha = 0.027870894746092358, reg_lambda = 0.009357684241550774,
#                    learning_rate = 0.09116640670855684, min_child_samples = 45)
#for l2 in [3.022,3.023,3.024,3.025,3.026,3.027,3.028,3.029, 3.030]:
clf = CatBoostClassifier(iterations = 1100, verbose=False, random_state=0, l2_leaf_reg=3.0)
#clf = BaggingClassifier(n_estimators=100,base_estimator=clf,random_state=0)
#clf = AdaBoostClassifier(base_estimator=clf, n_estimators=10, random_state=0)
#clf = RandomForestClassifier(base_estimator=clf, n_estimators=101, random_state = 0)
#clf = VotingClassifier(estimators=[('l1',clf1), 
#                                   ('l2',clf2), 
#                                    ('l3',clf3),
#                                    ('l4',clf4)
#                                  ], voting ='soft')
scores = 1+cross_val_score(clf, X, Y, cv = 5, scoring = 'neg_log_loss')
print(scores)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
clf.fit(X,Y)

[0.56057099 0.56385565 0.57041415 0.56208754 0.565843  ]
Accuracy: 0.5646 (+/- 0.0068)


BaggingClassifier(base_estimator=<catboost.core.CatBoostClassifier object at 0x7f5073d1f090>,
                  n_estimators=100, random_state=0)

In [229]:
# from bayes_opt import BayesianOptimization

# def lgb_eval(lambda_l2,lambda_l1,min_child_samples, lr,num_leaves=50, max_depth=-1):
#     params = {
#         #'is_unbalance': True,
#         "num_leaves" : int(num_leaves),
#         "max_depth" : int(max_depth),
#         "reg_alpha" : lambda_l2,
#         "reg_lambda" : lambda_l1,
#         "n_jobs" : 20,
#         "min_child_samples" : int(min_child_samples),
#         "learning_rate" : lr,
#         #"verbosity" : -1,
#         #"force_row_wise" : True,
#         "random_state" : 0
#     }
#     cll = LGBMClassifier(**params)
#     scores = 1+cross_val_score(cll, X, Y, cv = 4, scoring = 'neg_log_loss')
#     return scores.mean()


# lgbBO = BayesianOptimization(lgb_eval, {
#                                                 'lambda_l2': (0.0, 0.05),
#                                                 'lambda_l1': (0.0, 0.05),
#                                                 'min_child_samples': (15, 41),
#                                                 'lr' : (0.09, 0.11),
#                                                 #'max_depth': (5, 75),
#                                                 'num_leaves': (45, 65)
#                                                 })

# lgbBO.maximize(n_iter=100, init_points=5)
# print(lgbBO.max)

# from hpsklearn import HyperoptEstimator
# from hpsklearn import any_classifier, xgboost_classification
# from hpsklearn import any_preprocessing
# from hyperopt import tpe

# X_train, X_test, y_train, y_test = \
# train_test_split(X, Y, random_state=1, test_size=0.33)

# ############################################################################
# # Build and fit a regressor
# # =========================

# ...
# # define search
# # model = HyperoptEstimator(classifier=xgboost_classification('cla'), preprocessing=any_preprocessing('pre'), algo=tpe.suggest)
# # model.fit(X_train, y_train)
# # acc = model.score(X_test, y_test)
# # print("Accuracy: %.3f" % acc)
# # # summarize the best model
# # print(model.best_model())

# from catboost import Pool, cv

# import hyperopt

# def hyperopt_objective(params):
#     model = CatBoostClassifier(
#         l2_leaf_reg=4,
#         learning_rate=0.0650733105002455,
#         iterations=500,
#         eval_metric='Accuracy',
#         random_seed=42,
#         verbose=False,
#         loss_function='Logloss',
#         min_child_samples = 18,
#         max_depth = params['max_depth']
#     )
    
#     cv_data = cv(
#         Pool(X, Y),
#         model.get_params()
#     )
#     best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
#     return 1 - best_accuracy # as hyperopt minimise

# # def hyperopt_objective(params):
# #     model = LGBMClassifier(
# #         reg_lambda=params['reg_lambda'],
# #         reg_alpha=params['reg_alpha'],
# #         num_leaves=int(params['num_leaves'])+1,
# #         learning_rate=params['learning_rate'],
# #         metric='auc',
# #         random_seed=0,
# #         verbose=-1
# #     )
    
# #     best_accuracy = np.max(cross_val_score(model, X, Y, cv = 4, scoring = 'roc_auc'))
    
# #     return 1 - best_accuracy # as hyperopt minimise

# from numpy.random import RandomState

# params_space = {
# #    'num_leaves': hyperopt.hp.qloguniform('num_leaves', 0, 2.5, 1)
#     'max_depth': hyperopt.hp.qloguniform('max_depth', 0, 1.2, 1)
# #     'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-4, 5e-1),
# #     'reg_lambda': hyperopt.hp.uniform('reg_lambda', 1e-4, 5e-1),
# #     'reg_alpha': hyperopt.hp.uniform('reg_alpha', 1e-4, 5e-1)
# #     'min_child_samples' : hyperopt.hp.qloguniform('min_child_samples', 0, 4, 1)
# }

# trials = hyperopt.Trials()

# best = hyperopt.fmin(
#     hyperopt_objective,
#     space=params_space,
#     algo=hyperopt.tpe.suggest,
#     max_evals=50,
#     trials=trials,
#     rstate=RandomState(123)
# )
# print(best)

# import evalml
# from evalml import AutoMLSearch
# from evalml.model_family import ModelFamily
# X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, Y, problem_type='binary', test_size=.2)
# automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary', objective='auc', 
#                       allowed_model_families = [ModelFamily.RANDOM_FOREST, ModelFamily.XGBOOST, ModelFamily.CATBOOST, ModelFamily.LIGHTGBM],
#                       max_batches=50, ensembling=True)
# automl.search(data_checks='disabled')




# import h2o
# from h2o.estimators import H2OGradientBoostingEstimator
# h2o.init()
# Y_a = Y.astype(int)
# X_a = pd.DataFrame(X)
# X_a['target'] = Y_a
# data_train = X_a

# data_train_h2o=h2o.H2OFrame(data_train)

# data_train_h2o['target']=data_train_h2o['target'].asfactor() 

# # Build and train the model:
# model = H2OGradientBoostingEstimator(## more trees is better if the learning rate is small enough 
#   ## here, use "more than enough" trees - we have early stopping
#   ntrees = 10000,                                                            

#   ## smaller learning rate is better (this is a good value for most datasets, but see below for annealing)
#   learn_rate = 0.01,                                                         

#   ## early stopping once the validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
#   stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "log_loss", 

#   ## sample 80% of rows per tree
#   sample_rate = 0.8,                                                       

#   ## sample 80% of columns per split
#   col_sample_rate = 0.8,                                                   

#   ## fix a random number generator seed for reproducibility
#   seed = 1234,                                                             

#   ## score every 10 trees to make early stopping reproducible (it depends on the scoring interval)
#   score_tree_interval = 10, nfolds=5, max_depth=3)   ## Instantiating the class

# model.train(x=data_train_h2o.names[:-1],y=data_train_h2o.names[-1], training_frame=data_train_h2o)

# # Eval performance:
# model.model_performance()

# # Extract feature interactions:
# model.feature_interaction()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,25 mins 42 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.3
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_anish_l22tmy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.792 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%

Interaction Depth 0: 


,,interaction,gain,fscore,wfscore,average_wfscore,average_gain,expected_gain,gain_rank,fscore_rank,wfscore_rank,avg_wfscore_rank,avg_gain_rank,expected_gain_rank,average_rank,average_tree_index,average_tree_depth
0,,0,92005.562742,1138.0,587.512334,0.516267,80.848473,70912.697202,1.0,1.0,2.0,49.0,51.0,1.0,17.500000,685.941125,1.357645
1,,1,37124.743713,393.0,146.095702,0.371745,94.464997,27740.037603,5.0,7.0,8.0,85.0,36.0,5.0,24.333333,795.053435,1.600509
2,,2,22677.064385,440.0,237.859729,0.540590,51.538783,17898.328603,7.0,5.0,5.0,46.0,96.0,7.0,27.666667,982.290909,1.120455
3,,3,51167.649911,688.0,294.333477,0.427810,74.371584,32314.057552,3.0,3.0,3.0,70.0,53.0,3.0,22.500000,656.151163,1.335756
4,,4,15729.151815,163.0,54.283067,0.333025,96.497864,12230.181731,11.0,17.0,20.0,103.0,34.0,10.0,32.500000,1023.190184,1.595092
5,,5,15283.203488,279.0,73.811698,0.264558,54.778507,10156.385458,13.0,10.0,17.0,127.0,90.0,16.0,45.500000,898.007168,1.666667
6,,6,640.381342,27.0,4.429756,0.164065,23.717827,388.751784,130.0,102.0,139.0,158.0,144.0,129.0,133.666667,1500.740741,1.703704
7,,7,17654.585296,306.0,125.513630,0.410175,57.694723,11944.814570,8.0,8.0,9.0,75.0,81.0,11.0,32.000000,830.656863,1.343137
8,,8,3246.769270,116.0,41.606087,0.358673,27.989390,1475.906110,45.0,26.0,30.0,91.0,135.0,77.0,67.333333,1048.232759,1.310345
9,,9,12643.176238,124.0,45.225444,0.364721,101.961099,10659.938556,16.0,24.0,26.0,88.0,29.0,14.0,32.833333,1016.290323,1.564516



See the whole table with table.as_data_frame()

Interaction Depth 1: 


...


198 Split Value Histogram: 


,,split_value,count
0,,0.5,1



199 Split Value Histogram: 


,,split_value,count
0,,0.5,14
1,,1.5,2


[,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ]

In [240]:
# pipeline = automl.get_pipeline(122)
# pipeline.fit(X_train, y_train)
# print(pipeline.score(X_holdout, y_holdout, ["auc"]))
# clf=pipeline
# print(1-scores)
# clf.fit(X,Y)
raise Exception

#model.model_performance()

Exception: 

In [ ]:
feature_names = ['delta1', 'not_going_ratio', 'attended', 'distance', 'eaf', 'min_delta_ratio', 'invited',
                 'maybe', 'enf', 'emf']
feature_names += ["bcou"+str(i) for i in range(11)]
feature_names += ["wc"+str(i) for i in range(30)]
feature_names += ['train_count', 'test_count', 'delta2', 'delta3', 'not_going', 'gender', 'age', 'fr_with_org']
feature_names += ['yts', 'mts', 'dts', 'wts']
feature_names += ['yms', 'mms', 'dms', 'wms']
feature_names += ['ytd', 'mtd', 'dtd', 'wtd']
feature_names += ["tcou"+str(i) for i in range(11)]
feature_names += ["tcit"+str(i) for i in range(11)]
feature_names += ["lang"+str(i) for i in range(11)]
feature_names += ['tz','sum_wc' 'we1', 'we2', 'we3', 'lc', 'dc', 'no_fri']

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=13)

#params = { 'max_depth':20, 'n_estimators':201, 'max_features':gbmf, 'random_state':0 }
params = { 'max_depth':20, 'n_estimators':201, 'max_features':rfmf, 'random_state':0 , 'max_samples':0.7}
#params = { 'iterations':250, 'learning_rate':0.1, 'depth':10, 'eval_metric': 'Accuracy', 'random_state':0, 'verbose':100}
#params = { 'estimators' : [('rf',clf0), ('gb',clf1)], 'weights' : weights }

#reg = CatBoostClassifier(**params)
reg = RandomForestClassifier(**params)
#reg = GradientBoostingClassifier(**params)
#reg = VotingClassifier(**params)
reg.fit(X_train, y_train)

'''
mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(reg.staged_predict(X_test)):
    test_score[i] = reg.loss_(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, reg.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')
fig.tight_layout()
plt.show() '''

feature_importance = reg.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(17, 17))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, np.array(feature_names)[sorted_idx])
plt.title('Feature Importance (MDI)')

result = permutation_importance(reg, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(result.importances[sorted_idx].T,
            vert=False, labels=np.array(feature_names)[sorted_idx])
plt.title("Permutation Importance (test set)")
fig.tight_layout()
plt.show()

In [249]:
ts = {}
inv = {}
num_bt = {}
for index, row in test.iterrows():
    ts[(row["biker_id"], row["tour_id"])] = row["timestamp"]
    inv[(row["biker_id"], row["tour_id"])] = row["invited"]
    num_bt[(row["biker_id"], row["tour_id"])] = index
    
test_Y = clf.predict_proba(test_X)
test_Y.shape

(2690, 2)

In [250]:
train_bikers_set = np.array(train["biker_id"].drop_duplicates())
bikers_out = []
tours_out = []
for biker1 in tqdm(test_bikers_set):
    idx = np.where(biker1==test["biker_id"]) 
    tour = list(test["tour_id"].loc[idx]) # for each unique biker in test data get all the events  
    score = {}
    for tou in tour:
        #fe = make_feature(biker1, tou, ts[(biker1, tou)], inv[(biker1, tou)]).reshape(1,-1)
        #fe = [make_feature(biker1, tou, ts[(biker1, tou)], inv[(biker1, tou)])]
        #fe = scaler.transform(fe)
#         s1 = clf0.predict_proba(fe)[0][1]
#         s2 = clf1.predict_proba(fe)[0][1]
#         #print(s1, s2)
#         rf_wt = 0.5
#         score[tou] = s1*rf_wt + s2*(1-rf_wt)
        #print(clf.predict_proba(fe)[0])
        #s = clf.predict_proba(fe)[0][0]
        
        #s = clf.predict_proba(fe)[0][1]        
        
        s = test_Y[num_bt[(biker1, tou)]][1]
        score[tou] = s
    #print(score)
    tour.sort(key = lambda x : score[x], reverse = True)
    tour = " ".join(tour) # list to space delimited string
    bikers_out.append(biker1)
    tours_out.append(tour)
    

100%|██████████| 297/297 [00:00<00:00, 1324.68it/s]


In [251]:
sample_submission =pd.DataFrame(columns=["biker_id","tour_id"])
sample_submission["biker_id"] = bikers_out
sample_submission["tour_id"] = tours_out
sample_submission.to_csv("output/submission165.csv",index=False) # download this file from /kaggle/working directory
print(sample_submission.shape)
sample_submission.head(10)

(297, 2)


,biker_id,tour_id
0,CG33145288,YU2519340 QX16813281 QP76364848 QR03717384 WX2683695 RQ49464820 RV...
1,EC61865653,QU48928340 YQ5403718 ST80624055 US0307239 SR2219501 QS25647788 RR1...
2,CC23946882,YV890078 YV6176695 RU03553114 QT97445719 SX31661369 UX1543591 SU88...
3,DC82478477,YW217712 SV43004825 QV19013260 TR67083677
4,EC46728549,RQ33183406 VP2394192 RU29072432 QS61307272 WW1676713 RT89566262 SS...
5,DJ43175229,WU7252209 TR54164270 XP6696202 XU675684 SY4293006 RU8676103 SW7910...
6,EA60379526,QT66528489 SS27788367 QX10747729 SX28664529 SX79292193 VW2757840
7,HB3889887,RP43455253 WT1936389 QW76393554 SX07386691 QP95206162 QV99561506
8,CJ45719859,RU98678508 SS69927458 RQ45977549 SS84785057 SQ17416613 WY5158097
9,BA17536864,QV99663431 RW31716338 SQ50441220 QY9858305 RX05037878 ST99428374 X...


In [ ]:
#np.set_printoptions(formatter={'float': '{: 0.6f}'.format}, linewidth=200, suppress=True)
pd.DataFrame(X).to_csv("Data/x_1.csv", header=False)
# old_x = pd.read_csv("Data/x.csv", header = None)
# old_x[4].values
#probabs.to_csv("Data/probabs.csv", header=False)

In [ ]:
tours_useful_h = pd.read_csv("Data/tours_useful.csv")
bikers_useful_h = pd.read_csv("Data/bikers_useful.csv")
fig, axs = plt.subplots(1, 1, 
                        figsize =(10, 7),  
                        tight_layout = True) 
  
axs.hist(np.array(bikers_useful_h['latitude']), bins = 30) 
  
# Show plot 
plt.show() 

In [ ]:
#tours[tours["tour_id"] == "TP58829330"]
#X_train.shape
latlist = []
longlist = []
for tid, row in tours.items():
    if row[6] != '':
        latlist.append((float(row[6])))
    if row[7] != '':
        longlist.append((float(row[7])))
        
pd.DataFrame(longlist)[0].describe()

In [ ]:
dire = "Data"
biker=pd.read_csv(dire + "/bikers_useful.csv")
tours=pd.read_csv(dire + "/tours_useful.csv")
biker["latitude"] = biker["latitude"].replace(0, 3)
biker["longitude"] = biker["longitude"].replace(0, 100)
tours["latitude"] = tours["latitude"].fillna(3)
tours["longitude"] = tours["longitude"].fillna(100)
biker.to_csv(dire + "/bikers_useful.csv", index = False)
tours.to_csv(dire + "/tours_useful.csv", index = False)
tours.head()

In [ ]:
# locat = geolocator.reverse("41.25007, -69.9963491")
# str(locat).split(',')
for index, row in train.iterrows():
    bid = row['biker_id']
    tse = row['timestamp']
    print(get_lang(bid.upper()), bid.upper())

In [ ]:
top_10_loc = {}
gg.union(top_10_countries)
for coun in gg:
    nam = pycountry.countries.get(alpha_2=coun)
    name = 0.0
    if nam != None:
        name = nam.name
    else:
        continue
    loca = geolocator.geocode(name)
    if loca == None:
        print(name)
        continue
    top_10_loc[coun] = [loca.latitude, loca.longitude]
top_10_loc

In [ ]:
# biker_aba= pd.read_csv("temp/bikers_useful.csv")
# biker_aba.head()
# for index, row in train.iterrows():
#     b1 = row["biker_id"]
#     b2 = tours[row["tour_id"]][0]
#     print(b2 in biker_net)
b = []
for bike1 in tqdm(test_bikers_set):
    idxa = np.where(bike1==test["biker_id"]) 
    tourd = list(test["tour_id"].loc[idxa]) # for each unique biker in test data get all the events  
    for toud in tourd:
        b.append(toud)
b=np.array(b)
pd.DataFrame(b).to_csv("Data/test_tids.csv", header=None)

In [ ]:
tours['SY28440935'][2] == ''
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
#pd.DataFrame(test_X).to_csv("Data/test_x.csv", header=None)
nam = pycountry.countries.get(alpha_2='CA')
loca = geolocator.geocode(nam.name)
print(loca.latitude, loca.longitude)